Ön hazırlık (kütüphane kurulumu)

In [3]:
# Gerekli kütüphaneleri yükleyelim (Colab hücresine)
!pip install cirq qiskit qutip numpy matplotlib --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.8/670.8 kB 37.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 430.5/430.5 kB 28.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 80.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.8/31.8 MB 48.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 80.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 75.4 MB/s eta 0:00:00


1. Cirq ile Kuantum Fourier Dönüşümü

In [4]:
import cirq
import numpy as np

def qft_cirq(qubits):
    n = len(qubits)
    circuit = cirq.Circuit()

    for i in range(n):
        circuit.append(cirq.H(qubits[i]))
        for j in range(i + 1, n):
            theta = np.pi / (2 ** (j - i))
            circuit.append(cirq.CZ(qubits[j], qubits[i]) ** (1 / (2 ** (j - i))))
    # Qubitleri ters sıraya koymak için SWAP
    for i in range(n // 2):
        circuit.append(cirq.SWAP(qubits[i], qubits[n - i - 1]))

    return circuit

# 2 qubit QFT devresi
qubits = cirq.LineQubit.range(2)
circuit = qft_cirq(qubits)

# Simülasyon
sim = cirq.Simulator()
result = sim.simulate(circuit)
print("Son durum vektörü (|ψ⟩):\n", np.round(result.final_state_vector, 3))
print("\nCirq QFT Devresi:")
print(circuit)


Son durum vektörü (|ψ⟩):
 [0.5+0.j 0.5+0.j 0.5+0.j 0.5+0.j]

Cirq QFT Devresi:
0: ───H───@───────────×───
          │           │
1: ───────@^0.5───H───×───


2. Qiskit ile Kuantum Fourier Dönüşümü

In [5]:
from qiskit import QuantumCircuit, Aer, execute
from qiskit.visualization import plot_bloch_multivector
import numpy as np

def qft_qiskit(qc, n):
    for i in range(n):
        qc.h(i)
        for j in range(i + 1, n):
            qc.cp(np.pi / (2 ** (j - i)), j, i)
    # SWAP ile qubit sıralamasını ters çevir
    for i in range(n // 2):
        qc.swap(i, n - i - 1)
    return qc

# 2 qubit QFT devresi
qc = QuantumCircuit(2)
qft_qiskit(qc, 2)
print(qc.draw())

# Simülasyon
sim = Aer.get_backend('statevector_simulator')
result = execute(qc, sim).result()
state = result.get_statevector()
print("\nSon durum vektörü:\n", np.round(state, 3))


ImportError: cannot import name 'Aer' from 'qiskit' (/usr/local/lib/python3.12/dist-packages/qiskit/__init__.py)

3. QuTiP ile Kuantum Fourier Dönüşümü

In [6]:
from qutip import *
import numpy as np

def qft_qutip(n):
    """n-qubit QFT unitary operator"""
    N = 2 ** n
    omega = np.exp(2j * np.pi / N)
    QFT = np.zeros((N, N), dtype=complex)
    for j in range(N):
        for k in range(N):
            QFT[j, k] = omega ** (j * k) / np.sqrt(N)
    return Qobj(QFT)

# 2 qubit QFT
n = 2
Uqft = qft_qutip(n)

# Giriş durumu |01⟩
psi_in = tensor(basis(2,0), basis(2,1))
psi_out = Uqft * psi_in

print("QFT Çıkışı |ψ_out⟩:\n", np.round(psi_out.full(), 3))
print("\nQFT Unitary Matrisi:\n", np.round(Uqft.full(), 3))


TypeError: incompatible dimensions [[4], [4]] and [[2, 2], [1]]